# Семинар 4 - практика. Группировка

Анализируем датасет с информацией об атлетах участвовавших в Олимпийских играх (**olymp.csv**)

Данные содержат следующие признаки:
- **`ID`** - уникальный номер спортсмена
- **`Name`** - имя атлета
- **`Sex`** - пол (мужской - M,  женский - F)
- **`Age`** - возраст атлета
- **`Height`** - рост (в см)
- **`Weight`** - вес (в кг)
- **`Team`** - cтрана
- **`NOC`** - код страны от Олимпийского комитета (3 буквы)
- **`Games`** - год и сезон Олимпийских игр
- **`Year`** - год Олимпийских игр
- **`Season`** - сезон (Зимние - Winter,  Лето - Summer)
- **`City`** - город проведения Олимпийских игр
- **`Sport`** - спортивная дисциплина
- **`Event`** - категория спортивной дисциплины
- **`Medal`** - достоинство медали (Золото - Gold, Серебро - Silver, Бронза - Bronze, Без медали - NaN)


In [1]:
import pandas as pd

# определите свой путь к таблице
olmp = pd.read_csv('../../data/olymp.csv')

## Задание 1. 

Найдите год проведения Олимпиады, в который средний возраст атлетов был минимальный

**Решение**
- Сгруппируйте данные: 
    - столбец группировки - год/игра (**Year**/**Games**), 
    - столбец агрегирования - возраст (**Age**), 
    - метод агрегирования - среднее
- Отсортируйте данные по возрастанию с помощью `.sort_values()`

In [2]:
olmp.groupby(['Games'])['Age'].mean().sort_values().head()

Games
1984 Winter    23.443241
1896 Summer    23.580645
1980 Winter    23.656160
1980 Summer    23.704354
1976 Winter    23.739459
Name: Age, dtype: float64

**Ответ**: 1984

## Задание 2

Какой вид спорта (**Sport**) является самым многочисленным с точки зрения количества уникальных атлетов, принявших в нем участие?

In [3]:
olmp.groupby(['Sport'])['ID'].nunique().sort_values()

Sport
Aeronautics          1
Basque Pelota        2
Roque                4
Racquets             7
Croquet             10
                 ...  
Cycling           5819
Football          6161
Rowing            7687
Swimming          8765
Athletics        22071
Name: ID, Length: 66, dtype: int64

## Задание 3

Выясните, атлеты из какой страны (**Team**) завоевали наибольшее количество золотых медалей на Зимней Олимпиаде 1984 года

**Решение**
- Сделайте фильтрацию и оставьте только атлетов завоевавших золотые медали на Зимней Олимпиаде 1984 года
- На отфильтрованных данных постройте сводную таблицу: 
    - Индекс - названия стран (**Team**)
    - Колонка - **ID** атлета
    - В ячейках - уникальное количество атлетов из каждой страны. 

    Чтобы получить количество уникальных атлетов воспользуйтесь методом агрегирования `.nunique()`
- Отсортируйте данные по убыванию

In [4]:
olmp_filt = olmp[(olmp['Games'] == '1984 Winter') & (olmp['Medal']=='Gold')]
olmp_filt.groupby('Team')['ID'].nunique().sort_values(ascending=False)

Team
Soviet Union       27
Norway              6
East Germany        6
Sweden              5
United States       4
East Germany-1      4
East Germany-2      2
Soviet Union-1      2
Italy               2
Finland             2
Great Britain-1     2
West Germany-1      2
Switzerland         2
Canada              1
West Germany        1
Name: ID, dtype: int64

## Задание 4

Определите минимальное значение разницы между максимальным ростом и весом атлетов из определенной страны. В ответ запишите положительное целое число

**Решение**
- Создайте сводную таблицу: 
    - в рядах - короткое название страны (**NOC**), 
    - в колонках - рост (**Height**) и вес (**Weight**) атлетов
    - в ячейках - **максимальное** значение роста и веса по каждой стране
- В получившейся сводной таблице создайте новый признак, который будет рассчитан как разность между ростом и весом (по модулю)
- По новому признаку найдите минимум

In [5]:
olmp_4 = olmp.groupby('NOC')[['Weight', 'Height']].max()
olmp_4['w_h_diff'] = abs(olmp_4['Weight'] - olmp_4['Height'])

olmp_4['w_h_diff'].sort_values().head() #.iloc[0]

NOC
GUM    21.0
ESP    23.0
POL    25.0
NRU    27.0
VEN    28.0
Name: w_h_diff, dtype: float64

## Задание 5

Определите количество атлетов из Советского Союза (**Soviet Union**) чья доля успеха на олимпиаде не менее 0.5. 

Показатель доли успеха рассчитывается как частное (количество завоеванных медалей к количеству категорий спортивных дисциплин, в которых они приняли участие).

**Решение**
- Сделайте фильтрацию и оставьте только атлетов из Советского союза
- По отфильтрованным данным создайте сводную таблицу: в рядах - уникальный номер атлета (**ID**), в столбцах -  категория спортивной дисциплины (**Event**) и медали (**Medal**), в ячейках - количество категорий спортивных дисциплин и количество медалей для каждого атлета. Сохраните сгруппированные данные в новую переменную
- В получившемся датафрейме создайте новый признак доля успеха
- Сделайте фильтрацию по новому признаку

In [6]:
olmp_soviet = olmp[olmp['Team'] == 'Soviet Union']
olmp_soviet_grouped = olmp_soviet.groupby('ID')[['Event', 'Medal']].count()
olmp_soviet_grouped['success'] = olmp_soviet_grouped['Medal'] / olmp_soviet_grouped['Event']

sum(olmp_soviet_grouped['success'] >= 0.5)

1453

## Задание 6

Определите количество Олимпиад, на которых доля женщин больше, чем средняя доля их участия в целом

**Решение**
- Создайте сводную таблицу и сохраните ее в отдельную переменную: 
    - в рядах - олимпиада (**Games**), 
    - в колонке - уникальный номер атлета (**ID**), 
    - в ячейках  - количество уникальных атлетов в каждому году. 
- Создайте сводную таблицу только по женщинам: в рядах - олимпиада (**Games**), в колонке - уникальный номер атлетки (**ID**), в ячейках  - количество уникальных атлеток в каждому году. Сохраните сводную таблицу в переменную
- Найдите долю женщин в каждому году (найдите частное сводных таблиц). Получившуюся таблицу сохраните в новую переменную
- Сделайте фильтрацию и оставьте, только те Олимпиады, где доля женщин больше чем среднее значение доли их участия
- Посчитайте количество таких Олимпиад с метода `.count()`

In [7]:
olmp_games = olmp.groupby('Games')['ID'].nunique()
olmp_games_woman = olmp[olmp['Sex']=='F'].groupby('Games')['ID'].nunique()

woman_frac = olmp_games_woman / olmp_games
woman_frac_mean = woman_frac.mean()

olmp_games[woman_frac > woman_frac_mean].count()

np.int64(24)

## Задание 7

Создайте функцию `medal_weight`, которая на основании столбца **Medal**, определяет количество очков за медали разного достоинства:
- 3 - Gold
- 2 - Silver
- 1 - Bronze
- 0 - NaN

In [8]:
def medal_weight(x):
    if x == 'Gold':
        return 3
    elif x == 'Silver':
        return 2
    elif x == 'Bronze':
        return 1
    else:
        return x

## Задание 8

Датафрейм лежит в переменной **olmp**. Примените функцию созданную в прошлом задании к столбцу **Medal** и создайте новый признак **Medal_score** в датафрейме **olmp**.

In [9]:
olmp['Medal_score'] = olmp['Medal'].apply(medal_weight)

## Задание 9

Определите тип данных с точки зрения анализа в новом столбце **Medal_score**
- категориальный порядковый
- числовой непрерывный
- категориальный номинальный
- **числовой дискретный**
- категориальный номинальный бинарный

## Задание 10

Узнайте имя атлета (**Name**), который получил суммарно больше всего очков по новому столбцу **Medal_score**

In [10]:
olmp.groupby('Name')['Medal_score'].sum().sort_values(ascending=False)

Name
Michael Fred Phelps, II               77.0
Larysa Semenivna Latynina (Diriy-)    41.0
Nikolay Yefimovich Andrianov          34.0
Ole Einar Bjrndalen                   33.0
Paavo Johannes Nurmi                  33.0
                                      ... 
Hugo Dionisio Chila Ayov               0.0
Hugo De Grauwe                         0.0
Hugo Daini                             0.0
Hugo Cuna                              0.0
Hugo Gonzlez de Oliveira               0.0
Name: Medal_score, Length: 134732, dtype: float64